# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Fabian"
# Set the data and output directories
DATA_ROOT = '/Users/fabianmartinez/Desktop/DATA/TAIS_2024SP' 
OUTPUT_ROOT = '../../output1'
TRAIT = 'Essential-Thrombocythemia'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Colon_and_Rectal_Cancer_(COADREAD)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'CrawlData.ipynb',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Liver_Cancer_(LIHC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Cervical_Cancer_(CESC)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM

If no match is found, jump directly to GEO in Part 2.2

In [4]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [5]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [6]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [7]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [8]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [9]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [10]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [11]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [12]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [13]:
gender_col = 'gender'

In [14]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [15]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [16]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Essential-Thrombocythemia,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [17]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [18]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Essential-Thrombocythemia', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Essential-Thrombocythemia' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [19]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [20]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Fabian/Essential-Thrombocythemia/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [21]:
dataset = 'GEO'
trait_subdir = "Essential-Thrombocythemia"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE12295',
 'GSE32194',
 'GSE11003',
 'GSE174060',
 'GSE103176',
 'GSE19647',
 'GSE65161',
 'GSE18197',
 'GSE103237',
 'GSE103238',
 'GSE61629',
 'GSE42042',
 'GSE57793',
 'GSE159514',
 'GSE55976']

Repeat the below steps for all the accession numbers

In [22]:
cohort = accession_num = "GSE174060"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/fabianmartinez/Desktop/DATA/TAIS_2024SP/GEO/Essential-Thrombocythemia/GSE174060/GSE174060_family.soft.gz',
 '/Users/fabianmartinez/Desktop/DATA/TAIS_2024SP/GEO/Essential-Thrombocythemia/GSE174060/GSE174060_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [23]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Gene expression data from CD34+ BMMCs/PBMCs from MPN patients vs. healthy controls"
!Series_summary	"Philadelphia chromosome-negative myeloproliferative neoplasms (MPN) consist of primary myelofibrosis (PMF), polycythemia vera (PV), essential thrombocythemia (ET) and seconday myelofibrosis (SMF), comprising post-ET-MF(pET-MF) and post-PV-MF(pPV-MF)."
!Series_summary	"In this dataset, we compare the gene expression data of bone marrow or peripheral blood mononuclear cells (BMMCs/PBMCs) of CD34+ cells from MPN patients and healthy donors."
!Series_summary	""
!Series_overall_design	"36 samples (6 ET, 11 PV, 9 PMF, 4 secondary MF (SMF) patients and 6 healthy donors) were analyzed"


In [24]:
clinical_data.head()

,!Sample_geo_accession,GSM5285411,GSM5285412,GSM5285413,GSM5285414,GSM5285415,GSM5285416,GSM5285417,GSM5285418,GSM5285419,...,GSM5285437,GSM5285438,GSM5285439,GSM5285440,GSM5285441,GSM5285442,GSM5285443,GSM5285444,GSM5285445,GSM5285446
0,!Sample_characteristics_ch1,tissue: bone marrow,tissue: bone marrow,tissue: bone marrow,tissue: bone marrow,tissue: bone marrow,tissue: bone marrow,tissue: peripheral blood,tissue: bone marrow,tissue: bone marrow,...,tissue: peripheral blood,tissue: bone marrow,tissue: bone marrow,tissue: bone marrow,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood,tissue: peripheral blood
1,!Sample_characteristics_ch1,genotype: JAK2V617F,genotype: CALRins5,genotype: CALRdel52,genotype: MPLW515L,genotype: triple-negative,genotype: triple-negative,genotype: JAK2V617F,genotype: JAK2V617F,genotype: JAK2V617F,...,genotype: JAK2V617F,genotype: JAK2V617F,genotype: CALRdel52,genotype: JAK2V617F,genotype: N/A,genotype: N/A,genotype: N/A,genotype: N/A,genotype: N/A,genotype: N/A
2,!Sample_characteristics_ch1,age: 41,age: 53,age: 52,age: 47,age: 19,age: 33,age: 58,age: 76,age: 68,...,age: 71,age: 43,age: 56,age: 68,age: 29,age: 27,age: 28,age: 28,age: 32,age: 27
3,!Sample_characteristics_ch1,Sex: F,Sex: F,Sex: M,Sex: M,Sex: M,Sex: F,Sex: M,Sex: M,Sex: M,...,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: F,Sex: M,Sex: M,Sex: F,Sex: F
4,!Sample_characteristics_ch1,diagnosis: ET,diagnosis: ET,diagnosis: ET,diagnosis: ET,diagnosis: ET,diagnosis: ET,diagnosis: PV,diagnosis: PV,diagnosis: PV,...,diagnosis: pPV-MF,diagnosis: pPV-MF,diagnosis: pET-MF,diagnosis: pET-MF,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control,diagnosis: healthy control


In [25]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['tissue: bone marrow', 'tissue: peripheral blood'],
 1: ['genotype: JAK2V617F',
  'genotype: CALRins5',
  'genotype: CALRdel52',
  'genotype: MPLW515L',
  'genotype: triple-negative',
  'genotype: N/A'],
 2: ['age: 41',
  'age: 53',
  'age: 52',
  'age: 47',
  'age: 19',
  'age: 33',
  'age: 58',
  'age: 76',
  'age: 68',
  'age: 65',
  'age: 61',
  'age: 45',
  'age: 42',
  'age: 36',
  'age: 69',
  'age: 74',
  'age: 75',
  'age: 62',
  'age: 72',
  'age: 71',
  'age: 43',
  'age: 56',
  'age: 29',
  'age: 27',
  'age: 28',
  'age: 32'],
 3: ['Sex: F', 'Sex: M'],
 4: ['diagnosis: ET',
  'diagnosis: PV',
  'diagnosis: PMF',
  'diagnosis: pPV-MF',
  'diagnosis: pET-MF',
  'diagnosis: healthy control']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [26]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Essential-Thrombocythemia\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Essential-Thrombocythemia\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait i

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [27]:
is_gene_availabe = True
trait_row = 4
age_row = 2
gender_row = 3

trait_type = 'binary'

In [28]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [29]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type == 'diagnosis: ET':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'Sex: F':
        return 0
    elif gender_string == 'Sex: M':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'tissue: Hippocampus'
example_age = 'age: 23y'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(0, None, None)

In [30]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM5285411,GSM5285412,GSM5285413,GSM5285414,GSM5285415,GSM5285416,GSM5285417,GSM5285418,GSM5285419,GSM5285420,...,GSM5285437,GSM5285438,GSM5285439,GSM5285440,GSM5285441,GSM5285442,GSM5285443,GSM5285444,GSM5285445,GSM5285446
Essential-Thrombocythemia,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,41,53,52,47,19,33,58,76,68,65,...,71,43,56,68,29,27,28,28,32,27
Gender,0,0,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,1,1,0,0


### Genetic data preprocessing and final filtering

In [31]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM5285411,GSM5285412,GSM5285413,GSM5285414,GSM5285415,GSM5285416,GSM5285417,GSM5285418,GSM5285419,GSM5285420,...,GSM5285437,GSM5285438,GSM5285439,GSM5285440,GSM5285441,GSM5285442,GSM5285443,GSM5285444,GSM5285445,GSM5285446
ID,,,,,,,,,,,,,,,,,,,,,
TC01000001.hg.1,8.763194,9.444057,7.506218,10.137024,8.233619,7.749457,8.456729,8.563531,7.687238,7.945256,...,9.611053,11.483930,8.316019,10.314048,7.895399,8.357716,8.381425,7.559970,6.998370,7.722753
TC01000002.hg.1,5.431397,5.367823,5.563163,5.586412,5.539330,5.725849,6.084232,5.742522,5.320939,5.515826,...,5.656912,5.712754,5.334845,5.512505,5.778119,5.853299,5.352098,5.649065,5.634885,5.620705
TC01000003.hg.1,3.685363,3.970616,4.314971,4.508546,3.675396,4.103659,4.045691,4.001806,3.996599,3.882150,...,4.463492,4.067976,4.039990,3.645976,3.729595,3.981665,3.679181,4.026410,4.082079,4.037544
TC01000004.hg.1,6.351724,6.722208,7.226205,7.268921,6.263612,6.479180,5.852644,5.839868,6.526720,5.870487,...,6.441748,6.744685,6.306027,5.659950,6.069954,6.344466,6.134545,6.489107,7.568071,5.808890
TC01000005.hg.1,12.124011,11.728953,12.133326,11.918979,11.847020,11.838353,11.814903,10.311135,10.993554,10.733066,...,10.845970,11.803163,11.822833,10.681853,11.273564,10.984672,10.968623,10.894343,11.360254,11.173889


In [32]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['TC01000001.hg.1',
 'TC01000002.hg.1',
 'TC01000003.hg.1',
 'TC01000004.hg.1',
 'TC01000005.hg.1',
 'TC01000006.hg.1',
 'TC01000007.hg.1',
 'TC01000008.hg.1',
 'TC01000009.hg.1',
 'TC01000010.hg.1',
 'TC01000011.hg.1',
 'TC01000012.hg.1',
 'TC01000013.hg.1',
 'TC01000014.hg.1',
 'TC01000015.hg.1',
 'TC01000016.hg.1',
 'TC01000017.hg.1',
 'TC01000018.hg.1',
 'TC01000019.hg.1',
 'TC01000020.hg.1']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [33]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1', 'TC01000006.hg.1', 'TC01000007.hg.1', 'TC01000008.hg.1', 'TC01000009.hg.1', 'TC01000010.hg.1', 'TC01000011.hg.1', 'TC01000012.hg.1', 'TC01000013.hg.1', 'TC01000014.hg.1', 'TC01000015.hg.1', 'TC01000016.hg.1', 'TC01000017.hg.1', 'TC01000018.hg.1', 'TC01000019.hg.1', 'TC01000020.hg.1']\n"

If not required, jump directly to the gene normalization step

In [34]:
requires_gene_mapping = True

In [35]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// ENST00000456328 // DDX11L5 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 5 // 9p24.3 // 100287596 /// ENST00000456328 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102', 'ENST00000408384 // MIR1302-11 // microRNA 1302-11 // --- // 100422919 /// ENST00000408384 // MIR1302-10 // microRNA 1302-10 // --- // 100422834 /// ENST00000408384 // MIR1302-9 // microRN

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [36]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'probeset_id': ['TC01000001.hg.1', 'TC01000002.hg.1', 'TC01000003.hg.1', 'TC01000004.hg.1', 'TC01000005.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['11869', '29554', '69091', '160446', '317811'], 'stop': ['14409', '31109', '70008', '161525', '328581'], 'total_probes': [49.0, 60.0, 30.0, 30.0, 191.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287

In [37]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'gene_assignment'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [39]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

KeyboardInterrupt: 

In [40]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [41]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 36 samples.


In [42]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Essential-Thrombocythemia', the least common label is '1.0' with 6 occurrences. This represents 16.67% of the dataset.
The distribution of the feature 'Essential-Thrombocythemia' in this dataset is fine.

Quartiles for 'Age':
  25%: 39.75
  50% (Median): 54.5
  75%: 68.0
Min: 19.0
Max: 76.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 17 occurrences. This represents 47.22% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [43]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [44]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [63]:
from utils import *

In [64]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE174060'

In [65]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE174060,True,True,False,True,True,36,


In [66]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [67]:
merged_data.head()

,Essential-Thrombocythemia,Age,Gender
0,1.0,41.0,0.0
1,1.0,53.0,0.0
2,1.0,52.0,1.0
3,1.0,47.0,1.0
4,1.0,19.0,1.0


In [68]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [74]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values
#X

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [70]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

False

In [71]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [72]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

ValueError: Found array with 0 feature(s) (shape=(29, 0)) while a minimum of 1 is required by Lasso.

In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)